In [1]:
#Gaussian Binning

In [2]:
import pandas as pd
import numpy as np
from astropy import cosmology
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
from astropy.coordinates import SkyOffsetFrame
import astropy.units as unit
cosmo = FlatLambdaCDM(H0=100. * unit.km / unit.s / unit.Mpc, Om0=0.31) 
c = 299792.458

In [3]:
data = pd.read_csv('data_sph.csv')
errors=pd.read_csv('errors.csv')
print(errors)

          sigma_delta   sigma_vx   sigma_vy   sigma_vz
0            0.720856  273.71100  277.21840  268.93120
1            0.706480  271.19458  279.86185  271.08360
2            0.690903  269.07697  279.71906  277.78384
3            0.690298  269.30704  277.96274  285.90036
4            0.701487  272.24377  276.43842  291.06067
...               ...        ...        ...        ...
16777211     0.716592  260.04196  247.11726  269.15567
16777212     0.740938  258.68870  248.97945  274.63135
16777213     0.747300  258.68387  252.15308  278.55884
16777214     0.748521  260.50546  256.45053  279.52408
16777215     0.758744  262.70398  261.46896  277.21988

[16777216 rows x 4 columns]


In [4]:
print(data)

                sgx        sgy        sgz         vx         vy         vz  \
0        -500.00000 -500.00000 -500.00000  32.933980  22.674273 -10.697363   
1        -500.00000 -500.00000 -496.09375  36.657770  23.215712 -22.229534   
2        -500.00000 -500.00000 -492.18750  32.255474  22.005163 -42.015760   
3        -500.00000 -500.00000 -488.28125  32.745068  21.041777 -30.829197   
4        -500.00000 -500.00000 -484.37500  29.892162  26.177673 -33.858234   
...             ...        ...        ...        ...        ...        ...   
16777211  496.09375  496.09375  480.46875  35.339320  23.985851 -18.496784   
16777212  496.09375  496.09375  484.37500  33.905487  20.798400 -20.231531   
16777213  496.09375  496.09375  488.28125  32.573780  22.424404 -26.601788   
16777214  496.09375  496.09375  492.18750  37.626495  14.580416 -14.939850   
16777215  496.09375  496.09375  496.09375  42.471245  12.871185 -12.567743   

          Oph  Her  Pers   Sh     delta    rad_vel          RA 

In [5]:
print(errors)

          sigma_delta   sigma_vx   sigma_vy   sigma_vz
0            0.720856  273.71100  277.21840  268.93120
1            0.706480  271.19458  279.86185  271.08360
2            0.690903  269.07697  279.71906  277.78384
3            0.690298  269.30704  277.96274  285.90036
4            0.701487  272.24377  276.43842  291.06067
...               ...        ...        ...        ...
16777211     0.716592  260.04196  247.11726  269.15567
16777212     0.740938  258.68870  248.97945  274.63135
16777213     0.747300  258.68387  252.15308  278.55884
16777214     0.748521  260.50546  256.45053  279.52408
16777215     0.758744  262.70398  261.46896  277.21988

[16777216 rows x 4 columns]


In [6]:
#Setting the binning format
Dec_angular_bins = np.degrees(np.arccos(1 - 2 * np.arange(16 + 1) / 16))-np.degrees(np.pi/2)
ra_angular_bins=np.linspace(0, 360, 16 + 1)
def equal_volume_bins(R_min, R_max, n):
    r_min3 = R_min**3
    r_max3 = R_max**3
    r3_edges = np.linspace(r_min3, r_max3, n + 1)
    r_edges = r3_edges**(1/3)
    return r_edges

Distance_binning=equal_volume_bins(60,250,20)

In [7]:
#function for finding supernovae bin 


In [8]:
#Making the cosmology

from scipy import interpolate
import scipy.integrate as integrate

redshifts=np.array(np.arange(0,2.3,0.001))
chis=cosmo.comoving_distance(redshifts)
ztochi = interpolate.interp1d(redshifts, chis,kind='cubic')
chitoz = interpolate.interp1d(chis,redshifts,kind='cubic')
zbins=chitoz(Distance_binning)

def Hubble(_z):    
    return cosmo.H(_z ).to_value()

def InverseHubbleFLRW(z):
    hubbfl = c/Hubble(z)
    return hubbfl



#read the supernovae
SNdf = pd.read_table("Pantheon+SH0ES_zsel.dat", sep="\s+", usecols=['RA','DEC','zCMB','MU_SH0ES', 'MU_SH0ES_ERR_DIAG'])
SN1aH0up=SNdf.drop(SNdf[SNdf.zCMB > 0.15].index)
SN1aH0=SN1aH0up.drop(SN1aH0up[SN1aH0up.zCMB < 0.023].index)
SNdf=SN1aH0
RA= np.array(SNdf.RA)
Dec= np.array(SNdf.DEC)
reds=np.array(SNdf.zCMB)
fakenovae=np.stack((ztochi(reds),RA,Dec,reds),axis=1)

#finds their bin
R_edges=Distance_binning
Theta_edges=ra_angular_bins
Phi_edges=Dec_angular_bins

def find_single_bin(point):
#     Returns:
#         tuple of ints: (r_bin, theta_bin, phi_bin)
#             Bin indices, or -1 if coordinate is out of range.
    r, theta, phi = point[:3]  # use only the first three coordinates
    r_bin     = np.digitize(r,     R_edges) - 1
    theta_bin = np.digitize(theta, Theta_edges) - 1
    phi_bin   = np.digitize(phi,   Phi_edges) - 1
    # Check bounds
    if not (0 <= r_bin < len(R_edges) - 1):
        r_bin = -1
    if not (0 <= theta_bin < len(Theta_edges) - 1):
        theta_bin = -1
    if not (0 <= phi_bin < len(Phi_edges) - 1):
        phi_bin = -1
    return (r_bin, theta_bin, phi_bin)

In [9]:
sigmaradvel=np.sqrt(errors['sigma_vx']**2*data['sgx']**2 + errors['sigma_vy']**2*data['sgy']**2+errors['sigma_vz']**2*data['sgz']**2)/(np.sqrt(data['sgx']**2 +data['sgy']**2+data['sgz']**2))
sigmaradvel = np.nan_to_num(sigmaradvel, nan=0.0)
sigma_delta=np.array(errors['sigma_delta'])

In [10]:
#beginning of the randomization, everything above should be ran only once,
#everything below as many times as needed to reach convergence
import timeit,time
import random
vnl=175
randomization=1

In [11]:
rand_radvel=[]
rand_delta=[]
multistart=time.time()
if randomization==0:
    
    for i in range (len(data['delta'])):
# for i in range():
        covariance=np.array([[sigma_delta[i]**2,corr*sigma_delta[i]*sigmaradvel[i]],[corr*sigma_delta[i]*sigmaradvel[i],sigmaradvel[i]**2]])
#     print(covariance)
        rand_radvel.append(np.random.multivariate_normal([data.delta[i],data.rad_vel[i]],covariance)[1])
        rand_delta.append(np.random.multivariate_normal([data.delta[i],data.rad_vel[i]],covariance)[0])
multiend=time.time()
print(multiend -multistart)
if randomization==1:
    rand_delta= random.gauss(data['delta'],errors['sigma_delta'] )
    rand_radvel= random.gauss(data['rad_vel'],np.sqrt(sigmaradvel**2+vnl**2) )
    

0.00011801719665527344


In [12]:
startrandom=time.time()
rand_delta= random.gauss(data['delta'],errors['sigma_delta'] )
rand_radvel= random.gauss(data['rad_vel'],np.sqrt(sigmaradvel**2+vnl**2) )

lightdata = pd.DataFrame({
    'delta': rand_delta,
    'rad_vel': rand_radvel,
    'RA': data['RA'],
    'Dec': data['Dec'],
    'sgx': data['sgx'],
    'sgy': data['sgy'],
    'sgz': data['sgz'],
    'Distance': data['Distance']
})


In [13]:
start = time.time()
bins={}
integrandbin = np.zeros((len(Distance_binning)-1, len(ra_angular_bins)-1, len(Dec_angular_bins)-1))
scalefactor=np.zeros_like(integrandbin)
norm_integrandbin = np.zeros((len(Distance_binning)-1, len(ra_angular_bins)-1, len(Dec_angular_bins)-1))
norm_scalefactor=np.zeros_like(integrandbin)
for i in range(len(Distance_binning)-1):
    for j in range(len(ra_angular_bins)-1):
        for k in range(len(Dec_angular_bins)-1):
            condition=((lightdata.RA > ra_angular_bins[j]) & (lightdata.RA <= ra_angular_bins[j + 1]) &
            (lightdata.Dec > Dec_angular_bins[k]) & (lightdata.Dec <= Dec_angular_bins[k + 1]) & (lightdata.Distance > Distance_binning[i]) & (lightdata.Distance <= Distance_binning[i+1])
            )
            bins[(i,j,k)]= lightdata[condition]
            if i>0:
                integrandbin[i,j,k]=np.average((bins[i,j,k].delta/bins[i,j,k].Distance)*(Distance_binning[i]-Distance_binning[i-1]))
            else:
                integrandbin[i,j,k]=np.average((bins[i,j,k].delta/bins[i,j,k].Distance)*(Distance_binning[i]))
end = time.time()
print(end-start)


633.1969411373138


In [14]:
start = time.time()
for i in range(len(Distance_binning)-1):
    for j in range(len(ra_angular_bins)-1):
        for k in range(len(Dec_angular_bins)-1):
            scalefactor[i, j, k] = np.exp(-np.sum(integrandbin[i:(len(Distance_binning)), j, k]))
            max_in_the_bin=bins[i,j,k]['delta'].idxmax()
            max_loc = bins[i,j,k].loc[max_in_the_bin]
            Max_RA=max_loc.RA
            Max_Dec=max_loc.Dec + np.degrees(np.pi/2)
            Xszk=-np.cos(np.pi/180*Max_RA)*np.sin(np.pi/180*Max_Dec)
            Yzsk=-np.sin(np.pi/180*Max_RA)*np.sin(np.pi/180*Max_Dec)
            Zszk=np.cos(np.pi/180*Max_Dec)
            if i >=1:
                fi=(np.sin( np.pi*(bins[i,j,k].Distance-np.ones(len(bins[i,j,k].sgx))*Distance_binning[i-1])/(Distance_binning[i]-Distance_binning[i-1])))**2
            else:
                fi=(np.sin( np.pi*(bins[i,j,k].Distance-np.ones(len(bins[i,j,k].sgx)))/(Distance_binning[i])))**2
            W=(Xszk*np.cos(np.pi/180*bins[i,j,k].Dec +np.degrees(np.pi/2))*np.sin(np.pi/180*bins[i,j,k].RA)+Yzsk*np.sin(np.pi/180*bins[i,j,k].Dec +np.degrees(np.pi/2))*np.sin(np.pi/180*bins[i,j,k].RA)
              +Zszk*np.cos(np.pi/180*bins[i,j,k].Dec +np.degrees(np.pi/2)))*fi
            bins[(i,j,k)]=bins[(i,j,k)].assign(Jacobian=(scalefactor[i,j,k]**3)*(1+np.average(bins[i,j,k].delta)-W)  )           

            bins[(i,j,k)]=bins[(i,j,k)].assign(W=W)
end = time.time()
print(end-start)

7.377742052078247


In [15]:
Hq = {}
rhoq = {}
Oq = {}
for i in range(len(Distance_binning)-1):
    for j in range(len(ra_angular_bins)-1):
        for k in range(len(Dec_angular_bins)-1):
            numerator = 0
            denominator = 0
            for m in range(i + 1):
                numerator += np.sum(bins[(m, j, k)].rad_vel * np.absolute(bins[(m, j, k)].Jacobian) / (100 * bins[(m, j, k)].Distance))
                denominator += np.sum(np.absolute(bins[(m, j, k)].Jacobian))
            Hq[i,j,k] = 1 + numerator / denominator
            numerator = 0
            denominator = 0
            for m in range(i + 1):
                numerator += np.sum(bins[(m, j, k)].delta * np.absolute(bins[(m, j, k)].Jacobian))
                denominator += np.sum(np.absolute(bins[(m, j, k)].Jacobian))
            rhoq[i,j,k] = numerator / denominator
            Oq[i,j,k] = Hq[i,j,k]**2 - 1 -0.3*rhoq[i,j,k]

In [16]:
def comd_corr(supernova):
    binidentifier=find_single_bin(supernova)
    znova=supernova[3]
    chibin=[]
    if binidentifier[0]==0:
        chibin.append((1/Hq[0,binidentifier[1],binidentifier[2]])*integrate.quad(InverseHubbleFLRW ,0,znova)[0])
        totalcorr=(np.sum(chibin))
        fractcorr=totalcorr/ztochi(znova)    
    elif binidentifier[0]!=-1 and binidentifier[0]>0 :        
        for i in range(0, binidentifier[0]+1):
            if i==0:    
                chibin.append((1/Hq[0,binidentifier[1],binidentifier[2]])*integrate.quad(InverseHubbleFLRW ,0,zbins[i])[0])
            elif i==(binidentifier[0]-1):
                chibin.append((1/Hq[i,binidentifier[1],binidentifier[2]])*integrate.quad(InverseHubbleFLRW ,zbins[i-1],znova)[0])

            else:
                chibin.append((1/Hq[i,binidentifier[1],binidentifier[2]])*integrate.quad(InverseHubbleFLRW ,zbins[i-1],zbins[i])[0])

        totalcorr=(np.sum(chibin))
        fractcorr=totalcorr/(integrate.quad(InverseHubbleFLRW ,0,znova)[0])    
    else:

        for i in range(len(Distance_binning)-1):    
            if i==0:    
                chibin.append((1/Hq[i,binidentifier[1],binidentifier[2]])*integrate.quad(InverseHubbleFLRW ,0,zbins[i])[0])
            else:
                chibin.append((1/Hq[i,binidentifier[1],binidentifier[2]])*integrate.quad(InverseHubbleFLRW ,zbins[i-1],zbins[i])[0])
        totalcorr=(np.sum(chibin))+ integrate.quad(InverseHubbleFLRW,zbins[-1],znova)[0]
        fractcorr=totalcorr/ztochi(znova)

#     if fractcorr <0.7:
#         print(binidentifier,fractcorr)
#         print(chibin)
    return fractcorr

correctionsnovae=[]
for i in range(len(fakenovae)):
    correctionsnovae.append(comd_corr(fakenovae[i]))
    
endrandom=time.time()

In [18]:
print(endrandom-startrandom)

np.savetxt("correctionsnovae.csv", correctionsnovae, delimiter=",")


660.9578349590302
